In [ ]:
'''
    Special tokens: UNUSED3 tokens present in steps but, not encoded
'''

In [1]:
# Sets seeds for reproducability.

import torch
torch.manual_seed(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

In [2]:
from dataset import YouCookII
from dataset import YouCookIICollate
from torch.utils.data import DataLoader
from loss import *
from accuracy import *
from transformers import get_linear_schedule_with_warmup
from model import Model
from torch import nn

import numpy as np
import torch
import matplotlib.pyplot as plt

def train(model, num_actions_train=8, num_actions_valid=6, batch_size=4, epochs=25, lr=0.001, MAX_DETECTIONS=20):
    num_actions = num_actions_train
    
    train_dataset = YouCookII(num_actions, "/h/sagar/ece496-capstone/datasets/ycii")
    valid_dataset = YouCookII(num_actions_valid, "/h/sagar/ece496-capstone/datasets/ycii")
    
    train_size = len(train_dataset)
    valid_size = len(valid_dataset)
    
    print("Training Dataset Size: {}, Validation Dataset Size: {}".format(train_size, valid_size))
        
    collate = YouCookIICollate(MAX_DETECTIONS=MAX_DETECTIONS)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True, worker_init_fn=seed_worker)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate, drop_last=True, worker_init_fn=seed_worker)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, int(0.2*epochs), epochs)

    train_loss = np.zeros(epochs)
    valid_loss = np.zeros(epochs)
    
    train_accuracy = np.zeros(epochs)
    valid_accuracy = np.zeros(epochs)
    
    for epoch in range(epochs):
        model.train()
        
        epoch_loss = 0.
        num_batches = 0
        
        total = 0
        correct = 0
                
        for data in train_dataloader:
            _, bboxes, features, steps, entities, entity_count, _, _ = data
            
            # Zero out any gradients.
            optimizer.zero_grad()
            
            # Run inference (forward pass).
            loss_data, VG, RR = model(steps, features, bboxes, entities, entity_count)
            
            # Loss from alignment.
            loss_ = compute_loss_batched(loss_data)

            # Backpropagation (backward pass).
            loss_.backward()

            # Update parameters.
            optimizer.step()
            
            epoch_loss += loss_
            num_batches += 1
            
        # Scheduler update.
        scheduler.step()
        epoch_loss = epoch_loss / (num_batches * batch_size)
        
        # Save loss and accuracy at each epoch and plot.
        train_loss[epoch] = float(epoch_loss)
        train_accuracy[epoch] = get_alignment_accuracy(model, train_dataloader, batch_size, num_actions) 
        
        valid_loss[epoch] = get_alignment_loss(model, valid_dataloader, batch_size, num_actions)
        valid_accuracy[epoch] = get_alignment_accuracy(model, valid_dataloader, batch_size, num_actions)

        print("Epoch {} - Train Loss: {:.2f}, Validation Loss: {:.2f}, Train Accuracy: {:.2f}, Validation Accuracy: {:.2f}"
              .format(epoch + 1, train_loss[epoch], valid_loss[epoch], train_accuracy[epoch], valid_accuracy[epoch]))
    
    plt.figure()
    plt.plot(train_loss, label='train loss')
    plt.plot(valid_loss, label='valid loss')
    plt.legend()
    
    plt.figure()
    plt.plot(train_accuracy, label='train accuracy')
    plt.plot(valid_accuracy, label='valid accuracy')
    plt.legend()
    
    plt.show()
        
    return train_loss, valid_loss, train_accuracy, valid_accuracy, VG, loss_data, data

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(device)

In [ ]:
# Trainer.

num_actions_train = 8
num_actions_valid = 6
batch_size = 4
epochs = 100
lr = 1e-4

train_loss, valid_loss, train_accuracy, valid_accuracy, VG, loss_data, data = train(
    model, 
    num_actions_train=num_actions_train, 
    num_actions_valid=num_actions_valid, 
    batch_size=batch_size, 
    epochs=epochs, 
    lr=lr
)

In [4]:
# Evaluation.

from eval_fi import eval_all_dataset
eval_all_dataset(model, path="/h/sagar/ece496-capstone/datasets/fi")

[[['ground beef', 'salt', 'pepper', '1 egg', 'dijon mustard', 'bread crumbs', 'a bowl'], ['the ingredients', 'hand'], ['vegetable oil', 'a pan'], ['the meat patty', 'it'], ['2 slices of cheese', 'top'], ['they melt'], ['the burger'], ['some onion', 'the same pan'], ['some tomato sauce', 'the burger patty', 'the onions', 'a bun'], ['another bun on']]]
['in a bowl, add ground beef, salt, pepper, 1 egg, dijon mustard and bread crumbs. ', ' mix the ingredients with hand. ', ' Add vegetable oil in a pan. ', ' fry the meat patty in it. ', ' Put 2 slices of cheese on top. ', ' heat until they melt. ', ' transfer the burger aside. ', ' fry some onion in the same pan. ', ' Place some tomato sauce, the burger patty, the onions on a bun. ', ' cover with another bun on. ', ' ']
['[CLS]', 'in', 'a', 'bowl', ',', 'add', 'ground', 'beef', ',', 'salt', ',', 'pepper', ',', '1', 'egg', ',', 'di', '##jon', 'mustard', 'and', 'bread', 'cr', '##umb', '##s', '.', '[', 'unused', '##3', ']', 'mix', 'the', 'ing

Best IoU possible = 0.3893762757579662
Random Candidate IoU = 0.3376292451226239
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 6: add the cooked onions
This entity has no ground truth bounding box
--------------------------------------------------
Action 7: season with garlic salt, black pepper and chilli powder
Best IoU possible = 0.6763734817504883
Random Candidate IoU = 0.41253986954689026
Chosen Frame IoU: 0.6763734817504883
Best IoU possible = 0.8194847702980042
Random Candidate IoU = 0.2594963312149048
Chosen Frame IoU: 0.7165780067443848
This entity has no ground truth bounding box
--------------------------------------------------
Action 8: Give a good mix to combine them
Best IoU possible = 0.618232250213623
Random Candidate IoU = 0.41443637013435364
Chosen Frame IoU: 0.4117823541164398
--------------------------------------------------
Action 9: Shape th

Best IoU possible = 0.17735347151756287
Random Candidate IoU = 0.025044595822691917
Chosen Frame IoU: 0.1283475160598755
Best IoU possible = 0.8241345882415771
Random Candidate IoU = 0.05461464077234268
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 7: push the mixture down to the bottom of the jar
Best IoU possible = 0.8127436637878418
Random Candidate IoU = 0.019151194021105766
Chosen Frame IoU: 0.010426178574562073
This entity has no ground truth bounding box
--------------------------------------------------
Action 8: seal the jar
Best IoU possible = 0.8461233973503113
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0010274270316585898
--------------------------------------------------
Action 9: put it in room temperature for 3 days
Best IoU possible = 0.8665627241134644
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.03584675490856171
This entity has no ground truth bounding box
Mean Upper Bound IoU: 0.5205860137939453, Mean Random IoU: 0.05520165339

Random Candidate IoU = 0.047451990131493975
Chosen Frame IoU: 0.0
Best IoU possible = 0.3311355614409336
Random Candidate IoU = 0.17196815275584157
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 2: add some garlic oil
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: Add chopped onion, parsley, 1/2 spoon baking powder, coriander and sea salt
Best IoU possible = 0.6469965577125549
Random Candidate IoU = 0.10328702628612518
Chosen Frame IoU: 0.6469965577125549
Best IoU possible = 0.7298479080200195
Random Candidate IoU = 0.21431030333042145
Chosen Frame IoU: 0.4566173255443573
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 4: Add some sauce of your favourite
Best IoU possible = 0.7247858047485352
Random Candidate IoU = 0.004890283569693565
Chosen Frame IoU:

Best IoU possible = 0.767659604549408
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.40698832273483276
This entity has no ground truth bounding box
--------------------------------------------------
Action 5: cut the chicken into strips
Best IoU possible = 0.643816351890564
Random Candidate IoU = 0.4803982675075531
Chosen Frame IoU: 0.35790467262268066
This entity has no ground truth bounding box
--------------------------------------------------
Action 6: Add tomatos, olives, and chicken to the bowl of pasta
Best IoU possible = 0.5855317115783691
Random Candidate IoU = 0.32775482535362244
Chosen Frame IoU: 0.4023149013519287
Best IoU possible = 0.4600995182991028
Random Candidate IoU = 0.16468104720115662
Chosen Frame IoU: 0.41593652963638306
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 7: Add yogurt and cheese to a bowl
Best IoU possible = 0.8119426965713501
Random Candidate IoU = 0

...

Best IoU possible = 0.6943070888519287
Random Candidate IoU = 0.27869951725006104
Chosen Frame IoU: 0.4251793324947357
Best IoU possible = 0.5143947005271912
Random Candidate IoU = 0.22594113647937775
Chosen Frame IoU: 0.36380332708358765
--------------------------------------------------
Action 2: beat 6 eggs in a large cup
Best IoU possible = 0.5221183896064758
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.14940735697746277
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.05268469452857971
--------------------------------------------------
Action 3: Add milk to the cup
Best IoU possible = 0.27504922764894296
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.05670371909147216
Best IoU possible = 0.28417862500249663
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 4: Melt a little butter in the pan
Best IoU possible = 0.8442142605781555
Random Candidate IoU = 0.5186616778373718
Chosen Frame IoU: 0.42374300956

Best IoU possible = 0.5116598606109619
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 2: sprinkle sesame seeds on top of the rice
Best IoU possible = 0.21053092357137176
Random Candidate IoU = 0.05398339223875382
Chosen Frame IoU: 0.0
Best IoU possible = 0.4912942975357229
Random Candidate IoU = 0.06618377566337585
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 3: Flip the sheet over
Best IoU possible = 0.30055038288866487
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
This entity has no ground truth bounding box
--------------------------------------------------
Action 4: place tuna in a line on the sheet
Best IoU possible = 0.21428018808364868
Random Candidate IoU = 0.0010101812658831477
Chosen Frame IoU: 0.13047826290130615
Best IoU possible = 0.33287601274988265
Random Candidate IoU = 0.06337900459766388
Chosen Frame IoU: 0.17597298324108124
-----------------------------------------------

Best IoU possible = 0.5655730962753296
Random Candidate IoU = 0.2837687134742737
Chosen Frame IoU: 0.5655730962753296
This entity has no ground truth bounding box
This entity has no ground truth bounding box
--------------------------------------------------
Action 3: Add oil and an egg to the wok
Best IoU possible = 0.8238061666488647
Random Candidate IoU = 0.22684025490579127
Chosen Frame IoU: 0.4915103774403966
Best IoU possible = 0.6257783770561218
Random Candidate IoU = 0.4207547903060913
Chosen Frame IoU: 0.2783484160900116
This entity has no ground truth bounding box
--------------------------------------------------
Action 4: Add the chopped vegetables to the wok
Best IoU possible = 0.8131689429283142
Random Candidate IoU = 0.4900560975074768
Chosen Frame IoU: 0.704479992389679
Best IoU possible = 0.5178709030151367
Random Candidate IoU = 0.2967473864555359
Chosen Frame IoU: 0.3074105381965637
--------------------------------------------------
Action 5: stir
No entities detecte

0 8
the hot dogs
['the', 'hot', 'dogs']
102 109
103 105
the oil
['the', 'oil']
102 109
107 108
--------------------------------------------------
Action 1: mix lemon juice and milk
Best IoU possible = 0.544775128364563
Random Candidate IoU = 0.13147734073938583
Chosen Frame IoU: 0.06459014117717743
This entity has no ground truth bounding box
--------------------------------------------------
Action 2: Pour oil into a pot
Best IoU possible = 0.5050951838493347
Random Candidate IoU = 0.17968632280826569
Chosen Frame IoU: 0.19093391299247742
Best IoU possible = 0.6767905950546265
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.1709109991788864
--------------------------------------------------
Action 3: mix the cornmeal, flour, salt, sugar, and baking powder
Best IoU possible = 0.4874511957168579
Random Candidate IoU = 0.011871092952787876
Chosen Frame IoU: 0.043833330273628235
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no gro

Best IoU possible = 0.011388705112040043
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
--------------------------------------------------
Action 3: Add salt and black pepper to the shrimp
Best IoU possible = 0.02725665575089262
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.11760686309550451
Random Candidate IoU = 0.016640881076455116
Chosen Frame IoU: 0.003824853105470538
This entity has no ground truth bounding box
--------------------------------------------------
Action 4: Add an egg, salt, pepper, and cold water to a bowl of flour and baking powder
Best IoU possible = 0.2538499361489565
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.0
Best IoU possible = 0.8912841081619263
Random Candidate IoU = 0.0
Chosen Frame IoU: 0.06303951913140138
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
This entity has no ground truth bounding box
-----------------------------------------

In [ ]:
# Visualizer.

from visualizer import inference

YCII = "/h/sagar/ece496-capstone/datasets/ycii"
FI = "/h/sagar/ece496-capstone/datasets/fi"

VG, RR = inference(model, 27, 0, FI)

In [ ]:
# Saving and loading weights.

SAVE = False
LOAD = True

if SAVE:
    torch.save(model.state_dict(), "/h/sagar/ece496-capstone/weights/t1")
    
if LOAD:
    model.load_state_dict(torch.load("/h/sagar/ece496-capstone/weights/t1"))

In [ ]:
# Reload modules.

import importlib
import visualizer
import eval_fi
import model as mdl
import loss

importlib.reload(visualizer)
importlib.reload(eval_fi)
importlib.reload(mdl)
importlib.reload(loss)